<a href="https://colab.research.google.com/github/Doms-debug/ELM-algorithm-on-Iris-dataset/blob/main/ELM_Iris_Dominik_Czajka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ELM algorithm implementation on Iris dataset
++ evaluation of different activation functions


In [10]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.activations import mish

def elm_iris(X_train, y_train, X_test, y_test, warstwy_ukryte, activation_function='tanh'):
    # Zainicjuj losowo wagi wejściowe i bias
    warstwy_wejsciowe = X_train.shape[1]
    output_nodes = 4
    input_weights = np.random.uniform(-1, 1, size=(warstwy_wejsciowe, warstwy_ukryte))
    bias = np.random.uniform(-1, 1, size=(1, warstwy_ukryte))

    # Oblicz wyjścia warstwy ukrytej przy użyciu funkcji aktywacji tanh, sigmoid lub mish
    if activation_function == 'tanh':
      hidden_layer_output = np.tanh(np.dot(X_train, input_weights) + bias)
    elif  activation_function == 'sigmoid':
        hidden_layer_output = 1 / (1 + np.exp(-(np.dot(X_train, input_weights) + bias)))
    elif activation_function == 'mish':
           hidden_layer_output = mish(np.dot(X_train, input_weights) + bias).numpy()

    # Oblicz wagi wyjściowe przy użyciu np.linalg.pinv
    output_weights = np.dot(np.linalg.pinv(hidden_layer_output), y_train)

    # Przewidywanie etykiet klas przy użyciu wag wyjściowych i wyjść warstwy ukrytej
    if activation_function == 'tanh':
        hidden_layer_output_test = np.tanh(np.dot(X_test, input_weights) + bias)
    elif activation_function == 'sigmoid':
        hidden_layer_output_test = 1 / (1 + np.exp(-(np.dot(X_test, input_weights) + bias)))
    elif activation_function == 'mish':
      hidden_layer_output_test = 1 / (1 + np.exp(-(np.dot(X_test, input_weights) + bias)))

    predictions = np.dot(hidden_layer_output_test, output_weights)

    # Obliczanie dokładności
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(y_test, axis=1)
    accuracy = accuracy_score(true_labels, predicted_labels)

    return accuracy

iris = load_iris()
X, y = iris.data, iris.target

# One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

warstwy_ukryte = 1000

accuracy = elm_iris(X_train, y_train, X_test, y_test, warstwy_ukryte)
print(f"Dokładność modelu ELM dla zbioru Iris: {accuracy}")


for warstwy_ukryte in [50, 1000, 200, 500]:
    accuracy = elm_iris(X_train, y_train, X_test, y_test, warstwy_ukryte)
    print(f"Dokładność dla {warstwy_ukryte} ukrytych węzłów: {accuracy}")

# 2. Różne funkcje aktywacji
for activation_function in ['tanh', 'sigmoid', 'mish']:
    accuracy = elm_iris(X_train, y_train, X_test, y_test, warstwy_ukryte=100, activation_function=activation_function)
    print(f"Dokładność dla funkcji aktywacji {activation_function}: {accuracy}")

Dokładność modelu ELM dla zbioru Iris: 0.7333333333333333
Dokładność dla 50 ukrytych węzłów: 0.9666666666666667
Dokładność dla 1000 ukrytych węzłów: 0.9
Dokładność dla 200 ukrytych węzłów: 0.3
Dokładność dla 500 ukrytych węzłów: 0.7666666666666667
Dokładność dla funkcji aktywacji tanh: 0.6666666666666666
Dokładność dla funkcji aktywacji sigmoid: 0.6666666666666666
Dokładność dla funkcji aktywacji mish: 0.36666666666666664
